In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import thinkplot
from thinkdsp import read_wave
from matplotlib import pyplot as plt

edgePath = r'C:\Users\Vaclav\Documents\Diploma\Notebooks\Classifier\ImageBase\Edges'
linePath = r'C:\Users\Vaclav\Documents\Diploma\Notebooks\Classifier\ImageBase\Lines'

# root="TEST", formats=['png']
def plot_save(wave, root, formats,**options):
        fs = wave.frequencies()
        i = None
        fs = fs[:i]
        ts = wave.times()

        # make the array
        size = len(fs), len(ts)
        array = np.zeros(size, dtype=np.float)

        # copy amplitude from each spectrum into a column of the array
        for j, t in enumerate(ts):
            spectrum = wave.spec_map[t]
            array[:, j] = spectrum.amps[:i]
        
        thinkplot.pcolor(ts, fs, array, pcolor=True, contour=False, **options)
        thinkplot.save(root, formats, **options)

class EdgeFinder:
    def __init__(   self, image, filter_size=8, threshold1=35, threshold2=50,
                    rho = 1, theta = np.pi/180, min_votes = 10, min_line_length = 1, max_line_gap = 5,
                    cv_color=cv2.COLOR_RGB2GRAY, color=[255, 0, 0], thickness=4, mean_rate = 0.9, 
                    pos_intersect_ratio=10, neg_intersect_ratio=10):
                    
        self.orig_image = image
        self._cv_color = cv_color #cv2.COLOR_BGR2GRAY or cv2.COLOR_RGB2GRAY
        self.image = self._grayscale(image)
        self._filter_size = filter_size
        self._threshold1 = threshold1
        self._threshold2 = threshold2

        self._rho = rho # distance resolution in pixels of the Hough grid
        self._theta = theta # angular resolution in radians of the Hough grid
        self._min_votes = min_votes     # minimum number of votes (intersections in Hough grid cell)
        self._min_line_length = min_line_length #minimum number of pixels making up a line
        self._max_line_gap = max_line_gap    # maximum gap in pixels between connectable line segments

        self._color = color
        self._thickness = thickness
        self._pos_intersect_ratio = pos_intersect_ratio
        self._neg_intersect_ratio = neg_intersect_ratio
        self._line_list = []
        
        def onchangeThreshold1(pos):
            self._threshold1 = pos
            self._render()

        def onchangeThreshold2(pos):
            self._threshold2 = pos
            self._render()

        def onchangeFilterSize(pos):
            self._filter_size = pos
            self._filter_size += (self._filter_size + 1) % 2
            self._render()

        def onchangeRho(pos):
            self._rho = np.max([pos, 1])
            self._render()

        def onchangeTheta(pos):
            self._theta = np.max([pos*np.pi/180.0, np.pi/180.0])
            self._render()

        def onchangeMinVotes(pos):
            self._min_votes = pos
            self._render()

        def onchangeMinLineLength(pos):
            self._min_line_length = pos
            self._render()

        def onchangeMaxLineGap(pos):
            self._max_line_gap = pos
            self._render()

        self._render()

    def _grayscale(self, img):
        return cv2.cvtColor(img, self._cv_color)

    def _draw_lines(self, img, lines):
        try:
            for line in lines:
                for x1,y1,x2,y2 in line:
                    if abs(x2-x1) < 0.01:
                        continue
                    slope = round((y2-y1)/(x2-x1),1)
                    if np.isnan(slope) or np.isinf(slope):
                        continue
                    intercept = int(round(y1 - slope*x1))
                    if slope > 0 and ((x1+x2<img.shape[1]) or(intercept<img.shape[0]//self._pos_intersect_ratio)):
                        cv2.line(img, (x1, y1), (x2, y2), [0, 0, 255], self._thickness)
                    elif slope < 0 and ((x1+x2>img.shape[1]) or(intercept<(img.shape[0] + img.shape[0]//self._neg_intersect_ratio))):
                        cv2.line(img, (x1, y1), (x2, y2), [0, 0, 255], self._thickness)
                    elif abs(slope) < 0.5 or abs(slope) > 0.85:
                        cv2.line(img, (x1, y1), (x2, y2), [0, 0, 255], self._thickness)
                    else:
                        cv2.line(img, (x1, y1), (x2, y2), self._color, self._thickness)
        except Exception:
            print("No lines detected.")
                    
    def _hough_lines(self, img):
        """
        `img` should be the output of a Canny transform.
        Returns an image with hough lines drawn.
        """
        lines = cv2.HoughLinesP(img, self._rho, self._theta, self._min_votes, np.array([]), 
            minLineLength=self._min_line_length, maxLineGap=self._max_line_gap)
        self._line_list.append(lines)
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        self._draw_lines(line_img, lines)
        
        return line_img

    def threshold1(self):
        return self._threshold1

    def threshold2(self):
        return self._threshold2

    def filterSize(self):
        return self._filter_size
        
    def rho(self):
        return self._rho
        
    def theta(self):
        return self._theta
        
    def min_votes(self):
        return self._min_votes

    def min_line_length(self):
        return self._min_line_length

    def max_line_gap(self):
        return self._max_line_gap
        
    def edgeImage(self):
        return self._edge_img

    def smoothedImage(self):
        return self._smoothed_img

    def houghImage(self):
        return self._hough_img
    
    def getLinesList(self):
        return self._line_list

    def _render(self):
        self._smoothed_img = cv2.GaussianBlur(self.image, (self._filter_size, self._filter_size), sigmaX=0, sigmaY=0)
        self._edge_img = cv2.Canny(self._smoothed_img, self._threshold1, self._threshold2)
        self._hough_img = self._hough_lines(self._edge_img)
        # Draw the lines on the edge image
        self._hough_img = cv2.addWeighted(self._hough_img, 0.8, self.orig_image, 1., 0.)
        cv2.imwrite(os.path.join(edgePath, 'edges_'+filename),self._edge_img)    
        cv2.imwrite(os.path.join(linePath, 'lines_'+filename),self._hough_img)

In [2]:
## USE ONLY ONCE

#for filename in os.listdir('C:/Users/Vaclav/Documents/Diploma/Clean Signals/ETHALON_SIGNAL'):
#    wave = read_wave('C:/Users/Vaclav/Documents/Diploma/Clean Signals/ETHALON_SIGNAL/'+filename)
#    wave.normalize()
#    filename = os.path.splitext(filename)[0]
#    sig = wave.make_spectrogram(1024)
#    plot_save(sig, filename, ['png'])
#    filename = filename+'.png'
#    img = cv2.imread(filename)
#    edge_finder = EdgeFinder(img, color=[0, 0, 255], filter_size=7, threshold1=35, threshold2=50)

In [3]:
lines = []
for filename in os.listdir('C:/Users/Vaclav/Documents/Diploma/Clean Signals/ETHALON_SIGNAL'):
    filename = os.path.splitext(filename)[0]+'.png'
    img = cv2.imread(filename)
    edge_finder = EdgeFinder(img, color=[0, 0, 255], filter_size=7, threshold1=35, threshold2=50)
    lines.append(edge_finder.getLinesList())

In [4]:
print("Edge parameters:")
print("GaussianBlur Filter Size: %d" % edge_finder.filterSize())
print("Threshold1: %d" % edge_finder.threshold1())
print("Threshold2: %d" % edge_finder.threshold2())

print("Rho: %d" % edge_finder.rho())
print("Theta: %f" % edge_finder.theta())
print("Min Votes: %d" % edge_finder.min_votes())
print("min_line_length: %d" % edge_finder.min_line_length())
print("max_line_gap: %d" % edge_finder.max_line_gap())

Edge parameters:
GaussianBlur Filter Size: 7
Threshold1: 35
Threshold2: 50
Rho: 1
Theta: 0.017453
Min Votes: 10
min_line_length: 1
max_line_gap: 5


In [5]:
len(lines)

314

In [25]:
listOfLines = []
llines = []
i = 0
for i in list(range(0, 314, 1)):
    listOfLines.append(lines[i][0].tolist())
llines = listOfLines

for i in list(range(0, 314, 1)):
    for j in list(range(0, len(llines[i]), 1)):
        listOfLines[i][j] = llines[i][j][0]

In [51]:
df = pd.DataFrame(listOfLines[0], columns=['X1', 'Y1', 'X2', 'Y2'])
df[df['Y1'] == 795]

,X1,Y1,X2,Y2
5,1035,795,1040,795
6,1121,795,1123,795
16,1161,795,1167,795
20,978,795,990,795
21,929,795,936,795
28,1067,795,1079,795
36,873,795,875,795
38,952,795,968,795


In [45]:
df

,X1,Y1,X2,Y2
0,848,796,1198,796
1,1144,792,1194,792
2,951,793,1008,792
3,1009,791,1029,791
4,848,792,897,792
5,1035,795,1040,795
6,1121,795,1123,795
7,1568,199,1568,163
8,914,792,943,792
9,979,794,996,794
